In [15]:
import numpy as np
import cv2

import cv2
import numpy as np

def onMouse(event, x, y, flag, params):
    wname, img, gx,gy = params
    if event == cv2.EVENT_MOUSEMOVE:  # マウスが移動したときにx線とy線を更新する
        img2 = np.copy(img)
        h, w = img2.shape[0], img2.shape[1]
        cv2.line(img2, (x, 0), (x, h - 1), (255, 0, 0))
        cv2.line(img2, (0, y), (w - 1, y), (255, 0, 0))
        cv2.imshow(wname, img2)

    if event == cv2.EVENT_LBUTTONDOWN:  # レフトボタンをクリックしたとき、ptlist配列にx,y座標を格納する
            print('( %d, %d )' % (x, y),end="")
            dx = gx - x
            dy = gy - y
            if dx == 0 :
                print(dx,dy,0)
            else:
                if dx > 0:
                    print(180*np.arctan(dy/dx)/np.pi-90)
                else:
                    print(180*np.arctan(dy/dx)/np.pi+90)
            # cv2.circle(img, (x, y), 3, (0, 0, 255), 3)
            cv2.circle(img, (gx,gy), 3, (255,0,0), 3)
            cv2.imshow(wname, img)
            
img = cv2.imread("シルエット/17Spheric/17karami1o02_l.jpg",0)
color = cv2.imread("シルエット/17Spheric/17karami1o02_l.jpg",1)
mu = cv2.moments(img, False)
gx,gy= int(mu["m10"]/mu["m00"]) , int(mu["m01"]/mu["m00"])
print(gx,gy)
wname = "Angles"
cv2.namedWindow(wname)
npoints = 1
cv2.setMouseCallback(wname, onMouse, [wname, color, gx,gy])
cv2.imshow(wname, color)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

45 60
( 22, 71 )-115.55996517182382
( 21, 11 )-26.095424157388337


-1